In [5]:
!pip install pytesseract
!pip install nltk
!pip install requests
!pip install python-dotenv
!pip install spacy
!python -m spacy download en_core_web_sm

from PIL import Image
import pytesseract
# Install required libraries
import requests
import json
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import spacy
from spacy.tokens import Doc
from spacy.util import filter_spans

In [6]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Load the image from file
image_path = 'test4.jpeg'  # Replace with your image file path
image = Image.open(image_path)

In [7]:
extracted_text = pytesseract.image_to_string(image)

# Print the extracted text
print("Extracted Text:")
print(extracted_text)

In [8]:
def preprocess_text(text):
    # Tokenize into sentences
    sentences = sent_tokenize(text)
    print("\nTokenized Sentences:")
    print(sentences)
    
    # Tokenize into words
    words = word_tokenize(text)
    print("\nTokenized Words:")
    print(words)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    print("\nFiltered Words (Stopwords Removed):")
    print(filtered_words)

    # Calculate word frequency
    freq_dist = FreqDist(filtered_words)
    print("\nWord Frequencies:")
    for word, freq in freq_dist.most_common(10):
        print(f"{word}: {freq}")

    return {
        "sentences": sentences,
        "filtered_words": filtered_words,
        "freq_dist": freq_dist
    }

# Process the extracted text
processed_data = preprocess_text(extracted_text)
print(processed_data)

In [29]:
# Process the extracted text
processed_data = preprocess_text(extracted_text)
print(processed_data)

In [30]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

def summarize_with_gemini(api_key, extracted_text, custom_prompt):
    """
    Summarize text using Google's Gemini API
    """
    # Define the API endpoint - using the correct Gemini API URL
    endpoint = f"https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={api_key}"
    
    # Construct the payload according to Gemini API specifications
    payload = {
        "contents": [{
            "parts": [{
                "text": f"Instructions: {custom_prompt}\n\nText to analyze: {extracted_text}"
            }]
        }],
        "generationConfig": {
            "temperature": 0.7,
            "maxOutputTokens": 300,
            "topP": 0.8,
            "topK": 40
        }
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    try:
        # Send the POST request
        response = requests.post(endpoint, headers=headers, json=payload)
        
        # Check if the request was successful
        if response.status_code == 200:
            response_data = response.json()
            
            # Extract the generated text from the response
            if ('candidates' in response_data and 
                len(response_data['candidates']) > 0 and 
                'content' in response_data['candidates'][0] and 
                'parts' in response_data['candidates'][0]['content'] and 
                len(response_data['candidates'][0]['content']['parts']) > 0 and 
                'text' in response_data['candidates'][0]['content']['parts'][0]):
                
                return {
                    "success": True,
                    "summary": response_data['candidates'][0]['content']['parts'][0]['text']
                }
            else:
                return {
                    "success": False,
                    "error": "Unexpected response structure"
                }
        else:
            error_message = response.json().get('error', {}).get('message', 'Unknown error occurred')
            return {
                "success": False,
                "error": f"API request failed with status {response.status_code}: {error_message}"
            }
            
    except requests.exceptions.RequestException as e:
        return {
            "success": False,
            "error": f"Request failed: {str(e)}"
        }

# Example usage
if __name__ == "__main__":
    # Your API key
    api_key = os.getenv('GEMINI_API_KEY') # Replace with actual API key
    
    # Example text and prompt
    extracted_text = pytesseract.image_to_string(image)
    custom_prompt = "explain all the dishes?"
    
    # Call the function
    result = summarize_with_gemini(api_key, extracted_text, custom_prompt)
    
    # Handle the result
    if result.get("success", False):
        print("Summary:", result["summary"])
    else:
        print("Error:", result.get("error", "An unknown error occurred"))

In [31]:
def clean_and_normalize_text(text):
    """
    Advanced text cleaning and normalization using regex and NLTK
    """
    import re
    
    # Split text into lines and process each menu item
    lines = text.split('\n')
    food_items = []
    prices = []
    
    # Flag to identify menu section
    in_menu_section = False
    
    for line in lines:
        line = line.strip()
        
        # Skip empty lines
        if not line:
            continue
            
        # Start capturing items after the header
        if 'item Qty. Rate Total' in line:
            in_menu_section = True
            continue
            
        # Stop capturing when we hit totals section
        if 'Total Qty:' in line or 'Sub Total:' in line:
            in_menu_section = False
            continue
        
        if in_menu_section:
            # Try to extract menu items
            # Pattern: [Item name] [quantity] [rate] [total]
            parts = line.split()
            if len(parts) >= 4:  # Need at least item name, qty, rate, total
                try:
                    # Last number is the total price
                    total = float(parts[-1])
                    # Remove the last 3 numbers (qty, rate, total) to get item name
                    item_name = ' '.join(parts[:-3])
                    # Clean up any special characters
                    item_name = re.sub(r'[^\s]', '', item_name)
                    
                    if item_name and len(item_name) > 2:
                        food_items.append(item_name.title())
                        prices.append(total)
                except ValueError:
                    # Handle cases like "HYDERABADI MURG) BIRYANI"
                    if 'BIRYANI' in line:
                        # Special handling for biryani line
                        biryani_parts = [p for p in parts if p.replace('.', '').isdigit()]
                        if biryani_parts:
                            total = float(biryani_parts[-1])
                            item_name = ' '.join([p for p in parts if not p.replace('.', '').isdigit()])
                            item_name = re.sub(r'[^\s]', '', item_name)
                            food_items.append(item_name.title())
                            prices.append(total)
    
    return {
        'food_items': food_items,
        'prices': prices
    }

def generate_summary(processed_text):
    """
    Generate a descriptive summary of the food items and bill
    """
    items = processed_text['food_items']
    prices = processed_text['prices']
    
    if not items:
        return "No food items found in the bill."
    
    summary = "The bill includes "
    if len(items) == 1:
        summary += f"{items[0]}"
    elif len(items) == 2:
        summary += f"{items[0]} and {items[1]}"
    else:
        summary += ", ".join(items[:-1]) + f", and {items[-1]}"
    
    if prices:
        total = sum(prices)
        summary += f". The total amount is Rs. {total:.2f}"
    
    return summary

def extract_food_entities_with_spacy(text):
    """
    Use spaCy NER to identify food items and related entities
    """
    try:
        # Load the spaCy model
        nlp = spacy.load("en_core_web_sm")
        
        # Add custom food entity patterns
        food_patterns = [
            {"label": "FOOD", "pattern": [{"LOWER": "tandoori"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "chicken"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "biryani"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "roti"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "dal"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "tadka"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "spicy"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "lasooni"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "hyderabadi"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "murg"}]},
        ]
        
        # Add entity ruler to the pipeline
        ruler = nlp.add_pipe("entity_ruler", before="ner")
        ruler.add_patterns(food_patterns)
        
        # Process the text
        doc = nlp(text)
        
        # Return the identified food entities
        food_entities = [ent.text for ent in doc.ents if ent.label_ == "FOOD"]
        
        return food_entities
    
    except OSError:
        print("Error: The spaCy English model is not installed.")
        print("Please run: !python -m spacy download en_core_web_sm")
        return []

# Process the bill text to extract food items
food_items = extract_food_entities_with_spacy(extracted_text)
print("Food items identified using spaCy NER:")
for item in food_items:
    print(f"- {item}")

In [32]:
def apply_rule_based_classification(text):
    """
    Apply rule-based token classification with regex patterns to identify domain-specific entities
    """
    import re
    from collections import defaultdict
    
    # Dictionary to store all recognized entities
    entities = defaultdict(list)
    
    # Define patterns for different entity types
    patterns = {
        'FOOD_ITEM': [
            r'(?:Tandoori|Lasooni|HYDERABADI|BIRYANI|Dal|Tadka|Roti|chicken|spicy)\s*[\w\s]*',  # Food items
            r'[A-Z][a-z]+\s+[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*'  # Capitalized multi-word items
        ],
        'PRICE': [
            r'(?:Rs\.?|₹)?\s*\d+(?:[,.]\d{1,2})?',  # Indian currency format
            r'\d+\.\d{2}'  # Decimal price format
        ],
        'QUANTITY': [
            r'\b\d+\b(?!\.\d+)',  # Simple numbers like 1, 2, 3
            r'\d+\s*x',  # Format: 2x
            r'x\s*\d+',  # Format: x2
            r'\d+\s+(?:pcs|items?|pieces)'  # Format: 2 items, 3 pieces
        ],
        'DATE': [
            r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}',  # DD/MM/YYYY or MM/DD/YYYY
            r'\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{2,4}'  # 20 May 18
        ],
        'TIME': [
            r'\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM|am|pm)?'  # 12:55, 12:55 PM
        ],
        'ORDER_NUMBER': [
            r'(?:Order|Invoice|Bill)(?:\s+#|:|\s+Number:?)\s*[A-Za-z0-9-]+',  # Invoice Number: IN001001259
            r'#\s*[A-Za-z0-9-]+'  # #12345
        ],
        'CONTACT': [
            r'(?:\+\d{1,3}\s*)?(?:\(\d{3,4}\)\s*|\d{3,4}[-\s])\d{3,4}[-\s]\d{4}',  # Phone number formats
            r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'  # Email
        ],
        'ADDRESS': [
            r'\d+/\d+\s+\w+[-\s]\s*\d+',  # 11/2 Sector- 37
            r'[A-Z][a-z]+[-\s]\s*\d+'  # Faridabad- 121003
        ]
    }
    
    # Apply each pattern and extract matches
    for entity_type, pattern_list in patterns.items():
        for pattern in pattern_list:
            matches = re.finditer(pattern, text)
            for match in matches:
                # Get the matched text and its position
                start, end = match.span()
                matched_text = match.group()
                
                # Some post-processing and validation
                if entity_type == 'FOOD_ITEM' and len(matched_text) < 3:
                    continue  # Skip very short food items
                
                if entity_type == 'PRICE' and not any(c.isdigit() for c in matched_text):
                    continue  # Ensure prices contain digits
                
                # Store the entity with its position
                entities[entity_type].append({
                    'text': matched_text,
                    'start': start,
                    'end': end
                })
    
    return entities

def enhance_ner_with_rules(text):
    """
    Combine spaCy NER with rule-based classification to get the best of both approaches
    """
    # Get standard NER results
    try:
        nlp = spacy.load("en_core_web_sm")
        
        # Add custom entity patterns
        ruler = nlp.add_pipe("entity_ruler", before="ner")
        
        # Define patterns for food items and other domain-specific entities
        patterns = [
            {"label": "FOOD", "pattern": [{"LOWER": "tandoori"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "chicken"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "biryani"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "roti"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "dal"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "tadka"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "spicy"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "lasooni"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "hyderabadi"}]},
            {"label": "FOOD", "pattern": [{"LOWER": "murg"}]},
            {"label": "QUANTITY", "pattern": [{"IS_DIGIT": True}]},
            {"label": "PRICE", "pattern": [{"SHAPE": "ddd.dd"}]},
            {"label": "PRICE", "pattern": [{"SHAPE": "d,ddd.dd"}]},
        ]
        
        ruler.add_patterns(patterns)
        doc = nlp(text)
        
        spacy_entities = {
            "FOOD": [],
            "CARDINAL": [],
            "MONEY": [],
            "DATE": [],
            "TIME": [],
            "ORG": [],
            "PERSON": [],
        }
        
        # Extract entities recognized by spaCy
        for ent in doc.ents:
            if ent.label_ in spacy_entities:
                spacy_entities[ent.label_].append({
                    'text': ent.text,
                    'start': ent.start_char,
                    'end': ent.end_char
                })
    
    except Exception as e:
        print(f"Error in spaCy processing: {e}")
        spacy_entities = {}
    
    # Get rule-based entities
    rule_entities = apply_rule_based_classification(text)
    
    # Merge the results with priority to rule-based entities
    combined_entities = {}
    
    # First add rule-based entities
    for entity_type, entities in rule_entities.items():
        combined_entities[entity_type] = entities
    
    # Then add spaCy entities that don't overlap with rule-based ones
    for entity_type, entities in spacy_entities.items():
        if entity_type not in combined_entities:
            combined_entities[entity_type] = []
        
        for entity in entities:
            # Check if this entity overlaps with any existing entity
            overlaps = False
            for existing_type in combined_entities:
                for existing_entity in combined_entities[existing_type]:
                    # Check for overlap in character spans
                    if (entity['start'] < existing_entity['end'] and 
                        entity['end'] > existing_entity['start']):
                        overlaps = True
                        break
                if overlaps:
                    break
            
            # Add if no overlap
            if not overlaps:
                combined_entities[entity_type].append(entity)
    
    return combined_entities

print("\nApplying rule-based token classification to receipt text:")
rule_based_entities = apply_rule_based_classification(extracted_text)
for entity_type, entities in rule_based_entities.items():
    if entities:
        print(f"\n{entity_type}:")
        for entity in entities:
            print(f"  • {entity['text']}")

print("\nCombining spaCy NER with rule-based classification:")
combined_entities = enhance_ner_with_rules(extracted_text)
for entity_type, entities in combined_entities.items():
    if entities:
        print(f"\n{entity_type}:")
        for entity in entities:
            print(f"  • {entity['text']}")

In [33]:
# Install required libraries for BERT model implementation
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install tqdm
!pip install pandas
!pip install matplotlib seaborn

In [34]:
# Import libraries for BERT implementation
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertConfig, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import random

# Set seed for reproducibility
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed()

In [ ]:
# Integrate cropping and OCR into a single function
def process_image_to_text(image_path):
    """
    Process an image: read, crop, and extract text
    """
    try:
        # Load the image
        image = Image.open(image_path)
        
        # Extract text using pytesseract
        extracted_text = pytesseract.image_to_string(image)
        
        # Clean the extracted text
        cleaned_text = clean_text(extracted_text)
        
        return {
            "original_text": extracted_text,
            "cleaned_text": cleaned_text,
            "success": True
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

def clean_text(text):
    """
    Clean and normalize text for BERT processing
    """
    # Remove special characters and normalize whitespace
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = re.sub(r'[^\w\s.,;:!?$%&()-]', '', text)  # Keep only alphanumeric and basic punctuation
    
    # Remove redundant line breaks
    text = re.sub(r'\n+', ' ', text)
    
    return text.strip()

# Test the function with your image
result = process_image_to_text(image_path)
if result["success"]:
    print("Original Extracted Text:")
    print(result["original_text"][:500], "...\n")
    print("Cleaned Text for BERT:")
    print(result["cleaned_text"][:500], "...")
else:
    print(f"Error: {result['error']}")

In [ ]:
# Define dataset class for BERT
class ReceiptDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# For demonstration, create a dummy dataset or load real data if available
def prepare_sample_dataset():
    """
    Create a sample dataset for demonstration. 
    In a real scenario, you would load actual receipt data.
    """
    # Simulate different receipt types (e.g., restaurant, grocery, etc.)
    receipt_types = [
        "restaurant receipt", "grocery receipt", "cafe bill", 
        "hotel invoice", "retail purchase"
    ]
    
    # Generate data for demonstration
    texts = []
    labels = []
    
    # Let's create 100 sample records
    for i in range(100):
        # For demonstrating a multi-class problem (classify receipt type)
        label = i % len(receipt_types)
        
        # Generate synthetic text with characteristics of that receipt type
        if label == 0:  # restaurant
            text = f"Restaurant name: Sample Restaurant {i}\nServer: John\nTable: 12\nItems: Burger $10.99, Fries $4.99, Drink $2.99\nSubtotal: $18.97\nTax: $1.52\nTotal: $20.49\nTip: _______\nThank you!"
        elif label == 1:  # grocery
            text = f"Grocery Market {i}\n123 Main St\nItems:\nMilk $3.99\nBread $2.49\nEggs $4.29\nApples $5.99\nTotal: $16.76\nTax: $1.34\nTotal with tax: $18.10\nPaid with card: VISA **** 1234"
        elif label == 2:  # cafe
            text = f"Cafe Coffee {i}\nOrder #45678\nItems:\nLatte $4.50\nCroissant $3.25\nSubtotal: $7.75\nTax: $0.62\nTotal: $8.37\nThank you for visiting!"
        elif label == 3:  # hotel
            text = f"Hotel Paradise {i}\nGuest: Alex Smith\nRoom: 302\nCheck-in: 01/15/2023\nCheck-out: 01/18/2023\nRoom charge: $399.99\nResort fee: $45.00\nTax: $44.50\nTotal: $489.49\nThank you for staying with us!"
        else:  # retail
            text = f"Retail Store {i}\n555 Shopping Ave\nItems:\nT-shirt $19.99\nJeans $49.99\nSocks $8.99\nSubtotal: $78.97\nDiscount: -$10.00\nTax: $5.52\nTotal: $74.49\nPaid with: CREDIT"
        
        texts.append(clean_text(text))
        labels.append(label)
    
    return texts, labels, receipt_types

# Get or create dataset
texts, labels, class_names = prepare_sample_dataset()

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")
print(f"Class labels: {class_names}")

In [ ]:
# Define BERT Model for text classification
class BERTClassifier(nn.Module):
    def __init__(self, num_classes, pretrained_model="bert-base-uncased"):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        # Get BERT outputs
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        # Use the [CLS] token representation for classification
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        return logits

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_classes = len(class_names)
model = BERTClassifier(num_classes=num_classes)

# Create datasets
train_dataset = ReceiptDataset(X_train, y_train, tokenizer, max_len=256)
test_dataset = ReceiptDataset(X_test, y_test, tokenizer, max_len=256)

# Create data loaders
batch_size = 8
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)
test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=batch_size
)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

In [ ]:
from torch.optim import AdamW
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report
from transformers import get_linear_schedule_with_warmup

# Function to create a progress bar that works in any environment
def get_progress_bar(iterable, desc=None):
    # Try to use tqdm if available
    try:
        # Try regular tqdm first (avoid notebook issues)
        from tqdm import tqdm
        return tqdm(iterable, desc=desc)
    except (ImportError, Exception) as e:
        # Fallback to a simple iterator if tqdm is not available
        print(f"{desc}: {len(iterable)} batches")
        return iterable

# Training function with epoch-by-epoch visualization
def train_model(model, train_dataloader, test_dataloader, device, class_names, epochs=100):
    # Initialize optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    # Define loss function
    loss_fn = nn.CrossEntropyLoss()
    
    # To store training history
    training_stats = []
    epoch_train_losses = []
    epoch_val_losses = []
    epoch_train_accs = []
    epoch_val_accs = []
    
    # Set up plots for live updating
    plt.figure(figsize=(15, 5))
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    
    # Training loop
    for epoch in range(epochs):
        print(f"\n{'='*50}\nEpoch {epoch+1}/{epochs}\n{'='*50}")
        
        # Training phase
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0
        
        # Progress bar for training - using our robust implementation
        train_progress_bar = get_progress_bar(train_dataloader, desc="Training")
        
        for batch_idx, batch in enumerate(train_dataloader):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['label'].to(device)
            
            # Clear gradients
            model.zero_grad()
            
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            
            # Handle different output types (tensor vs object with logits)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs
            
            # Calculate loss
            loss = loss_fn(logits, labels)
            
            # Backward pass
            loss.backward()
            
            # Clip gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # Update weights
            optimizer.step()
            scheduler.step()
            
            # Track stats
            total_train_loss += loss.item()
            
            # Track accuracy
            _, predicted = torch.max(logits, 1)
            correct = (predicted == labels).sum().item()
            total_train_correct += correct
            total_train_samples += labels.size(0)
            
            # Regular print for non-tqdm progress
            if batch_idx % 10 == 0:  # Print every 10 batches
                print(f"Training Batch {batch_idx}/{len(train_dataloader)}: loss={loss.item():.4f}, acc={correct/labels.size(0):.4f}")
        
        # Calculate average training loss and accuracy for this epoch
        avg_train_loss = total_train_loss / len(train_dataloader)
        train_accuracy = total_train_correct / total_train_samples
        
        print(f"Training Loss: {avg_train_loss:.4f}")
        print(f"Training Accuracy: {train_accuracy:.4f}")
        
        # Evaluation phase
        model.eval()
        total_eval_loss = 0
        total_eval_correct = 0
        total_eval_samples = 0
        all_predictions = []
        all_true_labels = []
        
        # No gradients needed for evaluation
        with torch.no_grad():
            for batch_idx, batch in enumerate(test_dataloader):
                # Move batch to device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                labels = batch['label'].to(device)
                
                # Forward pass
                outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                
                # Handle different output types (tensor vs object with logits)
                logits = outputs.logits if hasattr(outputs, 'logits') else outputs
                
                # Calculate loss
                loss = loss_fn(logits, labels)
                total_eval_loss += loss.item()
                
                # Track accuracy
                _, predicted = torch.max(logits, 1)
                correct = (predicted == labels).sum().item()
                total_eval_correct += correct
                total_eval_samples += labels.size(0)
                
                # Collect predictions for metrics
                all_predictions.extend(predicted.cpu().numpy())
                all_true_labels.extend(labels.cpu().numpy())
                
                # Regular print for progress
                if batch_idx % 10 == 0:  # Print every 10 batches
                    print(f"Eval Batch {batch_idx}/{len(test_dataloader)}: loss={loss.item():.4f}, acc={correct/labels.size(0):.4f}")
        
        # Calculate average validation loss and accuracy
        avg_val_loss = total_eval_loss / len(test_dataloader)
        val_accuracy = total_eval_correct / total_eval_samples
        
        print(f"Validation Loss: {avg_val_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        
        # Store epoch statistics
        epoch_train_losses.append(avg_train_loss)
        epoch_val_losses.append(avg_val_loss)
        epoch_train_accs.append(train_accuracy)
        epoch_val_accs.append(val_accuracy)
        
        # Update plots
        epochs_range = list(range(1, epoch + 2))
        
        # Compute classification report for this epoch
        report = classification_report(all_true_labels, all_predictions, 
                                      target_names=class_names, zero_division=1)
        print("\nClassification Report:")
        print(report)
        
        # Save stats
        training_stats.append({
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'train_acc': train_accuracy,
            'val_acc': val_accuracy,
            'classification_report': report
        })
    
    # Save model
    model_path = os.path.join(os.getcwd(), 'bert_receipt_classifier.pt')
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")
    
    return training_stats

training_stats = train_model(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    device=device,
    class_names=class_names,
    epochs=100
)

In [ ]:
# Function to test model on new receipts
def classify_receipt_text(text, model, tokenizer, class_names):
    """
    Classify a receipt text using the trained BERT model
    """
    # Clean the text
    cleaned_text = clean_text(text)
    
    # Prepare input tensors
    encoding = tokenizer.encode_plus(
        cleaned_text,
        add_special_tokens=True,
        max_length=256,
        return_token_type_ids=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move tensors to device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask, token_type_ids)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    # Get predicted class and confidence
    predicted_class = class_names[predicted.item()]
    confidence = probabilities[0][predicted.item()].item()
    
    # Return all class probabilities for visualization
    all_probs = {class_name: prob.item() for class_name, prob in zip(class_names, probabilities[0])}
    
    return {
        'class': predicted_class,
        'confidence': confidence,
        'all_probabilities': all_probs
    }

# Test on new receipts from the test dataset
for i in range(5):  # Test 5 random samples
    idx = random.randint(0, len(X_test) - 1)
    text = X_test[idx]
    true_label = class_names[y_test[idx]]
    
    # Classify the receipt
    result = classify_receipt_text(text, model, tokenizer, class_names)
    
    print(f"\nSample {i+1}:")
    print(f"Text: {text[:150]}...")
    print(f"True label: {true_label}")
    print(f"Predicted label: {result['class']}")
    print(f"Confidence: {result['confidence']:.4f}")
    
    # Visualize all probabilities
    plt.figure(figsize=(10, 5))
    plt.bar(result['all_probabilities'].keys(), result['all_probabilities'].values())
    plt.title(f"Class Probabilities for Sample {i+1}")
    plt.xlabel("Class")
    plt.ylabel("Probability")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [2]:
import os
import re
import cv2
import torch
import pandas as pd
import pytesseract
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import torch.nn as nn

# 🔹 Step 1: OCR Setup
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# 🔹 Step 2: Extract Text from Cropped Receipts
def extract_text_from_images(folder_path):
    data = []
    for file in os.listdir(folder_path):
        if file.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            text = pytesseract.image_to_string(img)  # OCR extraction
            data.append({"filename": file, "text": text.strip()})
    return pd.DataFrame(data)

folder_path = r"C:\Users\nanda\OneDrive\Desktop\NLP-Mini-Project\Cropped_Receipts"
df = extract_text_from_images(folder_path)

# 🔹 Step 3: Automated Labeling Rules
def classify_text(text):
    text = text.strip()
    
    # Rule 1: Amount Detection ($XX.XX or XX.XX format)
    if re.search(r"\$\d+\.\d{2}|\b\d+\.\d{2}\b", text):
        return "total_amount"

    # Rule 2: Date Detection (MM/DD/YYYY or DD/MM/YYYY)
    if re.search(r"\b\d{1,2}/\d{1,2}/\d{4}\b", text):
        return "date"

    # Rule 3: Restaurant Name Detection (Known Chains)
    restaurant_keywords = ["McDonald's", "Subway", "Starbucks", "KFC", "Pizza Hut", "Domino's", "Burger King"]
    if any(kw.lower() in text.lower() for kw in restaurant_keywords):
        return "restaurant_name"

    # Rule 4: Item List Detection (Comma-separated items)
    if "," in text and len(text.split()) > 2:
        return "items"
    
    return "other"

df["category"] = df["text"].apply(classify_text)

# 🔹 Step 4: Save Auto-Labeled Data for BERT Training
df = df[df["category"] != "other"]  # Remove unclassified
df.to_csv("auto_labeled_receipt_data.csv", index=False)

print("✅ Auto-labeling complete! Data saved as auto_labeled_receipt_data.csv")

# 🔹 Step 5: Load Tokenizer & Prepare Data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

category_mapping = {cat: idx for idx, cat in enumerate(df["category"].unique())}
df["label"] = df["category"].map(category_mapping)

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

class ReceiptDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding="max_length",
            truncation=True, return_attention_mask=True, return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

train_dataset = ReceiptDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len=64)
test_dataset = ReceiptDataset(X_test.tolist(), y_test.tolist(), tokenizer, max_len=64)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
test_dataloader = DataLoader(test_dataset, sampler=RandomSampler(test_dataset), batch_size=8)

# 🔹 Step 6: Define BERT Model
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

    def forward(self, input_ids, attention_mask):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask).logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(num_classes=len(category_mapping)).to(device)

# 🔹 Step 7: Train the Model
def train_model(model, train_dataloader, epochs=100):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.CrossEntropyLoss()
    model.train()

    for epoch in range(epochs):
        total_train_loss, total_correct, total_samples = 0, 0, 0

        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training"):
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

        print(f"Epoch {epoch+1}: Loss = {total_train_loss/len(train_dataloader):.4f}, Accuracy = {total_correct/total_samples:.4f}")

train_model(model, train_dataloader, epochs=100)

# 🔹 Step 8: Evaluate the Model
def evaluate_model(model, test_dataloader):
    model.eval()
    total_correct, total_samples = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            logits = model(input_ids, attention_mask)
            preds = torch.argmax(logits, dim=1)

            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = total_correct / total_samples
    print(f"Test Accuracy: {acc:.4f}")

    # Fix: Get only the present labels in predictions
    present_labels = sorted(set(all_labels))  
    target_names = [k for k, v in category_mapping.items() if v in present_labels]

    print(classification_report(all_labels, all_preds, labels=present_labels, target_names=target_names))

evaluate_model(model, test_dataloader)

# 🔹 Step 9: Predict on New Receipt Text
def predict_category(text):
    model.eval()
    encoding = tokenizer.encode_plus(text, return_tensors="pt", max_length=64, padding="max_length", truncation=True)
    input_ids, attention_mask = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        pred = torch.argmax(logits, dim=1).cpu().numpy()[0]
    
    category = [k for k, v in category_mapping.items() if v == pred][0]
    return category

# Example Prediction
sample_text = "$14.89"
print(f"Predicted Category: {predict_category(sample_text)}")
